In [1]:
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

In [1]:
import os
import json

In [3]:
dataset_path = "instruction_dataset/acc_data/2d"
new_file_path = f"datasets/acc_data_2d_gpt.jsonl"
train_data = []

In [4]:
llama_format = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>
{user_question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{model_answer}<|eot_id|>
"""

In [7]:
for root, dirs, files in os.walk(dataset_path):
    for file_name in files:
        if file_name.endswith(".json"):
            file_path = os.path.join(root, file_name)
            with open(file_path, 'r') as json_file:
                try:
                    data = json.load(json_file)
                    train_data.append(data)  
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {file_path}: {e}")

print(f"Loaded {len(train_data)} JSON files.")

Loaded 265 JSON files.


In [8]:
with open(new_file_path, "w", encoding="utf-8") as new_file:
    for piece in train_data:
        for trial in piece:
            gpt_format = {
                "messages": [
                    {"role": "system", "content": trial["instruction"]},
                    {"role": "user", "content": trial["input"]},
                    {"role": "assistant", "content": trial["output"]}
                ]
            }
            new_file.write(json.dumps(gpt_format))
            new_file.write("\n")

print(f"Converted data saved to {new_file_path}.")

Converted data saved to datasets/acc_data_2d_gpt.jsonl.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open(new_file_path, "rb"),
  purpose="fine-tune"
)

FileObject(id='file-DVC33bkDoKd7dVvqA3qGuZ', bytes=7938335, created_at=1733255442, filename='datasets/acc_data_2d_gpt.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [11]:
client.fine_tuning.jobs.create(
  training_file="file-DVC33bkDoKd7dVvqA3qGuZ",
  model="gpt-4o-2024-08-06",
  hyperparameters={
    "n_epochs":2
  }
)

FineTuningJob(id='ftjob-DIh9osS0eSzOM4RDmdSahX5E', created_at=1733256031, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-bejkEelMUYAOQ31IkgcGQPTr', result_files=[], seed=325651572, status='validating_files', trained_tokens=None, training_file='file-DVC33bkDoKd7dVvqA3qGuZ', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [53]:
client.fine_tuning.jobs.retrieve("ftjob-DIh9osS0eSzOM4RDmdSahX5E").fine_tuned_model


'ft:gpt-4o-2024-08-06:personal::AaUNbC4p'

In [4]:
from openai import OpenAI
client = OpenAI()
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-DIh9osS0eSzOM4RDmdSahX5E", limit=10).data[0].message


'The job has successfully completed'

In [27]:
train_loss = []
log = client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-DIh9osS0eSzOM4RDmdSahX5E", limit=1600)
for i in range(1600):
    try:
        info = log.data[i].message
    except:
        break
    if '1590' in info:
        train_loss.append(float(info.split('=')[-1]))
        

In [31]:
train_loss.reverse()

1.1

In [32]:
import csv

data = zip(range(len(train_loss)), train_loss)

# Save to CSV
with open('gpt_ft_loss.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['epoch', 'loss'])  # Write header
    writer.writerows(data)             # Write data rows


In [ ]:
## suffix can be found in by after training is completed
model = client.fine_tuning.jobs.retrieve("ftjob-DIh9osS0eSzOM4RDmdSahX5E").fine_tuned_model